# Introduction

The Project is to build a model to improve the Zestimate residual error. 

   $ logerror=log(Zestimate)−log(SalePrice) $ 

“Zestimates” are Zillow's estimated home values. The model is to predict the difference between the Zillow's estimated home value, Zestimate, and the actual sale price. 

## Data

We have 4 files. We merge properties_2016.csv and train_2016.csv to make a training data which has 90275 data points(houses). For a test data, we merge properties_2017.csv and train_2017.csv. The test data has 77613 data points.  

   **properties_2016.csv**: a full list of real estate properties in three counties (Los Angeles, Orange and Ventura, 
                           California) data in 2016.

   **train_2016.csv**: all the transactions before October 15, 2016, plus some of the transactions after October 
                      15,2016. It contains parcel ID ,  transaction date and calculated log error . 

   **properties_2017.csv**: a full list of real estate properties in three counties (Los Angeles, Orange and Ventura,
                           California) data in 2017.

   **train_2017.csv**: all the transactions from Jan 1, 2017 to Sep 25, 2017. It can be used as a test dataset.



In [ ]:
#import a necessary module

import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import seaborn as sns

# read files

path_properties_2016 ='/Users/kimjisun/Documents/SpringBoard/Project 1/zillow/properties_2016.csv'
df_properties_2016 = pd.read_csv(path_properties_2016, low_memory=False)

path_train_2016 ='/Users/kimjisun/Documents/SpringBoard/Project 1/zillow/train_2016.csv'
df_train_2016 = pd.read_csv(path_train_2016 , parse_dates=["transactiondate"], low_memory=False)

path_properties_2017 ='/Users/kimjisun/Documents/SpringBoard/Project 1/zillow/properties_2017.csv'
df_properties_2017 = pd.read_csv(path_properties_2017, low_memory=False)

path_train_2017 ='/Users/kimjisun/Documents/SpringBoard/Project 1/zillow/train_2017.csv'
df_train_2017 = pd.read_csv(path_train_2017 , parse_dates=["transactiondate"],low_memory=False)

In [ ]:
# merge training and properties data for 2016 and 2017

df_train_2016 = pd.merge(df_train_2016, df_properties_2016, how='inner', on='parcelid')
df_test_2017 = pd.merge(df_train_2017, df_properties_2017, how='inner', on='parcelid')

In [ ]:
df_train_2016.shape

In [ ]:
df_test_2017.shape

In [ ]:
df_train_2016.head().transpose()

In [ ]:
df_train_2016.describe().transpose()

In [ ]:
df_train_2016.info()

In [ ]:
test = df_train_2016[df_train_2016.bathroomcnt !=df_train_2016.calculatedbathnbr]
test1 = test[test.bathroomcnt!=0.0]
test2 = test[test.calculatedbathnbr.isnull()]
test.shape
#test2[['bathroomcnt','calculatedbathnbr' ]]


# Data Wrangling

## Data Cleaning

I explored training data. 125 duplicated parcelid for 2016 data and 199 duplicated parcelid for 2017 data were found. However, it meant they were trasacted for more than twice for a year. so it didn't have any problem.



In [ ]:
df_train_2016.shape
sum(df_train_2016.duplicated('parcelid'))

In [ ]:
df_test_2017.shape
sum(df_train_2017.duplicated('parcelid'))

In [ ]:
# Check if negative values in each column. Two columns, logerror and longitude, have negative values which are reasonable. 

df_train_2016.describe().loc["min",:][ df_train_2016.describe().loc["min",:] < 0 ]

In [ ]:
# Check if columns with object type have unusual cells. 

# there are 5 columns with a object type

df_train_2016.loc[:, df_train_2016.dtypes == np.object].columns

In [ ]:
# check 5 columns to see if they have unnual cells. 

df_train_2016["hashottuborspa"].unique() , df_train_2016["propertycountylandusecode"].unique() , df_train_2016["propertyzoningdesc"].unique(), df_train_2016["fireplaceflag"].unique(), df_train_2016["taxdelinquencyflag"].unique()

## Missing Values

Let's check how many missing value each column has. 

We can find that 47 columns have missing values and 18 columns among them have more than 95% of missing values. 


In [ ]:
df_2016_missing = df_train_2016.isnull().sum(axis=0).reset_index()
df_2016_missing.columns = ["index",'missing_values']


df_2016_missing = df_2016_missing[df_2016_missing.missing_values != 0]
df_2016_missing['ratio'] =df_2016_missing['missing_values']/len(df_train_2016)

missing_values = df_2016_missing.sort_values(by='ratio', ascending=False )
missing_values

In [ ]:
len(missing_values[missing_values['ratio']>0.95])

In [ ]:
# Draw horizontal Bar plot for missing values

plt.rcdefaults()
fig, ax = plt.subplots(figsize=(12,18))

y_pos = np.arange(len(missing_values))


ax.barh(y_pos, missing_values['missing_values'],   color='green')
ax.set_yticks(y_pos)
ax.set_yticklabels(missing_values['index'].values)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Missing values')
ax.set_title('missing values in each column')

plt.show()





## Outliers

Let's draw a scatter plot on "logerror", then we can find that there are some outliers at the end of both sides. 

Our task in the project is to find where the zillow algorithm fails. These outliers means where the zillow algorithm fails the most. Thus, I will leave outliers just like that. 


In [ ]:
plt.scatter(range(df_train_2016.shape[0]), df_train_2016.sort_values(by='logerror')['logerror'])
plt.show()

# Data Storytelling

## Distribution of Logerror

We would check both logerror and absolute value of logerror. Logerror indicates wheather estimated house values has been underestimated or overestimated while absolute logerror tells us that how estimated house value is close to an actual house value. It seems like the distribution of logerror follows a normal distribution. 

In [ ]:
#Distribution of Logerror

plt.hist(df_train_2016.logerror,50)
plt.title("Distribution of Logerror")
plt.xlabel("Logerror")
plt.ylabel("Count")
plt.show()

In [ ]:
#Distribution of absolute Logerror

plt.hist(abs(df_train_2016.logerror),50)
plt.title("Distribution of absolute Logerror")
plt.xlabel("Logerror")
plt.ylabel("Count")
plt.show()

## Transaction dates

Let's check the distribution of transaction dates, there are fewer transactions after October

In [ ]:
#Distribution of transaction dates

trs_month = df_train_2016['transactiondate'].dt.month.value_counts()

plt.figure(figsize=(12,6))
sns.barplot(trs_month.index, trs_month.values, alpha=0.8,color ='blue' )

plt.xlabel('Month of transaction')
plt.ylabel('Count')
plt.show()



## Correlation

Let's check correlations to "logeror" to see how variables are related. Square of basement and year built are the top 2 variables which have high correlations. 


In [ ]:
corr = df_train_2016.corr()

corr_logerror = corr.loc["logerror",:] [ corr.loc["logerror",:].notnull() ].reset_index()
corr_logerror.columns = ["index",'corr']
corr_logerror_sorted = corr_logerror.sort_values(by='corr', ascending=False )[1:]



plt.rcdefaults()
fig, ax = plt.subplots(figsize=(12,18))

y_pos = np.arange(len(corr_logerror_sorted))


ax.barh(y_pos, corr_logerror_sorted['corr'],   color='green')
ax.set_yticks(y_pos)
ax.set_yticklabels(corr_logerror_sorted['index'].values)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Correlation')
ax.set_title('Correlation for logerror')

plt.show()



# Let's explore Variables


## Number of Bedroom

It seems logerror is high when number of bedroom is 4. 

In [ ]:
y= df_train_2016['logerror']
x= df_train_2016['bedroomcnt']
s= df_train_2016['bedroomcnt']

corr = np.corrcoef(x, y)[0, 1]
plt.figure(figsize=(5,5))
plt.title('check short_desc corr = %s'%corr)
plt.xlabel('bedroomcnt')
plt.ylabel('logerror')
plt.grid(True)
plt.scatter(x, y, alpha=0.5,s=s)
plt.show()

## Lattitude and longitude

In [ ]:
y= df_train_2016['latitude']
x= df_train_2016['longitude']
s=1
plt.figure(figsize=(5,5))

plt.xlabel('longitude')
plt.ylabel('latitude')
plt.grid(True)
plt.scatter(x, y, alpha=0.5,s=s)
plt.show()

##  Type of cooling system present in the home

There are 61494 missing values out of 90275. I filled missing values with 0 to see how logerror is related to type of cooling system.  Logerror is low when cooling system > 1

Type of cooling system is like below. 
1	Central, 
2	Chilled Water,
3	Evaporative Cooler,
4	Geo Thermal,
5	None,
6	Other,
7	Packaged AC Unit,
8	Partial,
9	Refrigeration,
10	Ventilation,
11	Wall Unit,
12	Window Unit

In [ ]:
df_train_2016['transaction_month'] = df_train_2016['transactiondate'].dt.month
train_y = df_train_2016['logerror'].values
cat_cols = ["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", "fireplaceflag", "taxdelinquencyflag"]
train_df = df_train_2016.drop(['parcelid', 'logerror', 'transaction_month']+cat_cols, axis=1)
feat_names = df_train_2016.columns.values

train_df.head(5)

In [ ]:
train_df['airconditioningtypeid'].describe()

In [ ]:
train_df.loc[train_df['airconditioningtypeid'].isnull(),'airconditioningtypeid']=0

In [ ]:
y= df_train_2016['logerror']
x= train_df['airconditioningtypeid']

s = 1
corr = np.corrcoef(x, y)[0, 1]
plt.figure(figsize=(5,5))
plt.title('check short_desc corr = %s'%corr)
plt.xlabel('airconditioningtypeid')
plt.ylabel('logerror')
plt.grid(True)
plt.scatter(x, y, alpha=0.5,s=s)

plt.show()

##  Architectural style of the home

There are 90014 missing values out of 90275. Missing values are filled with 0 .  Logerror is high when value is 0. Logerror is low for 261 non-missing values. Need to explore more for 261 values to see how logerror is low. 

In [ ]:
train_df['architecturalstyletypeid'].describe()

In [ ]:
train_df.loc[train_df['architecturalstyletypeid'].isnull(),'architecturalstyletypeid']=0

y= df_train_2016['logerror']
x= train_df['architecturalstyletypeid']

s = 1
corr = np.corrcoef(x, y)[0, 1]
plt.figure(figsize=(5,5))
plt.title('check short_desc corr = %s'%corr)
plt.xlabel('architecturalstyletypeid')
plt.ylabel('logerror')
plt.grid(True)
plt.scatter(x, y, alpha=0.5,s=s)

plt.show()

##  Finished living area below or partially below ground level

In [ ]:
train_df['basementsqft'].describe()

In [ ]:
df_train_2016[df_train_2016['basementsqft'].isnull()]['logerror'].describe()

In [ ]:
df_train_2016[df_train_2016['basementsqft'].isnull()]['logerror'].hist(bins=50)
plt.show()

In [ ]:
df_train_2016[~df_train_2016['basementsqft'].isnull()]['logerror'].hist(bins=50)
plt.show()

## Number of Bathrooms

In [ ]:
df_train_2016['bathroomcnt'].describe()

In [ ]:
y= df_train_2016['logerror']
x= train_df['bathroomcnt']

s = 1
corr = np.corrcoef(x, y)[0, 1]
plt.figure(figsize=(5,5))
plt.title('check short_desc corr = %s'%corr)
plt.xlabel('bathroomcnt')
plt.ylabel('logerror')
plt.grid(True)
plt.scatter(x, y, alpha=0.5,s=s)

plt.show()

## County in which the property is located

In [ ]:

train_df['regionidcounty'].unique()



In [ ]:
y= df_train_2016['logerror']
x= train_df['regionidcounty']

s = 1
corr = np.corrcoef(x, y)[0, 1]
plt.figure(figsize=(5,5))
plt.title('check short_desc corr = %s'%corr)
plt.xlabel('Recion County')
plt.ylabel('logerror')
plt.grid(True)
plt.scatter(x, y, alpha=0.5,s=s)

plt.show()

In [ ]:
train_df['regionidcounty'].hist()
plt.show()